In [1]:

import pandas as pd
import numpy as np
import string
import os
import sys
import shutil

main_ds = pd.read_json('STAR_val.json')
main = main_ds.loc[:, ['question_id','question','video_id','start','end','answer', 'choices']]

os.chdir('./datatables2')
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

dfs = []

for csv in csv_files:
    df = pd.read_csv(csv,index_col=None)
    dfs.append(df)
    
final_df = pd.concat(dfs, ignore_index=True)

#final_df = pd.read_csv('/Users/sanjivjha/Developer/IMP-OIC/STAR_eval/all_tested.csv')
final_df = final_df.drop_duplicates()
final_df.reindex()
final_df.head()

ds = final_df
ds = ds.dropna()
ds = ds.loc[:, ~ds.columns.str.contains('^Unnamed')]
ds.head()
ds.name = 'all'

/Users/sanjivjha/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
q_ids = ds['question_id']
print(len(q_ids))
for q in q_ids:
  que = ds.query("question_id == '"+q+"'")
  answer = que['answer'].values[0].lower().strip('the ').translate(str.maketrans('', '', string.punctuation))
  choices = dict()
  choice_string = ''
  options = main_ds.query("question_id == '"+q+"'")["choices"].values[0]
  for choice in options:
    choices.update({choice['choice_id']:choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)).strip()})
    choice_string += ' ('+str(choice['choice_id']+1)+')'+str(choice['choice'].lower().strip('the').translate(str.maketrans('', '', string.punctuation)))  
  OIC_answer = int(ds.query("question_id == '"+q+"'")["OIC_answer"].values[0])
  if choices[int(OIC_answer)-1] == answer:
    ds.loc[ds['question_id'] == q, 'Match'] = 'Correct'
    
  else:
   
    ds.loc[ds['question_id'] == q, 'Match'] = 'Wrong'


1140


In [3]:
interaction_ds = ds.loc[ds['question_id'].str.contains('Interaction')]
interaction_ds.name='Interaction'
interaction_ds.head()


,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
2,Interaction_T1_76,Which object was thrown by the person?,XO8NL,3.6,12.5,The pillow.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",In frame 0: man_fa9c wearing shirt_5f78.man_fa...,4,Which object was thrown by the person? Guess t...,Wrong
3,Interaction_T1_78,Which object was opened by the person?,OY3LS,2.5,10.1,The laptop.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",In frame 0: glove_ad46 on hand_7269.man_2437 w...,3,Which object was opened by the person? Guess t...,Wrong
4,Interaction_T1_101,Which object was put down by the person?,RG0KS,17.4,27.0,The box.,"[{'choice_id': 0, 'choice': 'The cup/glass/bot...",In frame 0: man_6529 wearing shirt_0e52.man_65...,4,Which object was put down by the person? Guess...,Wrong
5,Interaction_T1_137,Which object was closed by the person?,NUKJ0,19.4,26.6,The book.,"[{'choice_id': 0, 'choice': 'The book.', 'choi...",In frame 0: woman_af39 wearing shirt_1005.woma...,4,Which object was closed by the person? Guess t...,Wrong
6,Interaction_T1_171,Which object was put down by the person?,LBRYS,8.9,13.4,The shoe.,"[{'choice_id': 0, 'choice': 'The sandwich.', '...",In frame 0: man_4430 wearing shirt_1a40.man_44...,3,Which object was put down by the person? Guess...,Correct


In [4]:
sequence_ds = ds.loc[ds['question_id'].str.contains('Sequence')]
sequence_ds.name = 'Sequence'
sequence_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
3870,Sequence_T1_19,Which object did the person put down after the...,RNLTR,7.3,16.4,The shoe.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on man_b5d8 wearing...,3.0,Which object did the person put down after the...,Wrong
3871,Sequence_T1_53,Which object did the person throw after they p...,4GLAP,15.7,20.7,The shoe.,"[{'choice_id': 0, 'choice': 'The shoe.', 'choi...",The scene opens:\nFrom now on handle_8d7d on d...,2.0,Which object did the person throw after they p...,Wrong
3872,Sequence_T1_56,Which object did the person put down after the...,XO8NL,3.6,11.6,The towel.,"[{'choice_id': 0, 'choice': 'The clothes.', 'c...",The scene opens:\nFrom now on man_02df wearing...,3.0,Which object did the person put down after the...,Wrong
12974,Sequence_T2_444,Which object did the person open before they h...,3Q6N1,0.0,9.4,The laptop.,"[{'choice_id': 0, 'choice': 'The bag.', 'choic...",In frame 0: man_a815 sitting on chair_763e.man...,3.0,Which object did the person open before they h...,Correct
12984,Sequence_T2_849,Which object did the person close before they ...,3JCEI,8.3,15.1,The laptop.,"[{'choice_id': 0, 'choice': 'The laptop.', 'ch...",In frame 0: man_a899 wearing shirt_056d.man_a8...,1.0,Which object did the person close before they ...,Correct


In [5]:
prediction_ds = ds.loc[ds['question_id'].str.contains('Prediction')]
prediction_ds.name='Prediction'
prediction_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
13935,Prediction_T1_111,What will the person do next?,D1WYU,0.0,9.000,Throw the bag.,"[{'choice_id': 0, 'choice': 'Open the laptop.'...",In frame 0: man_9307 wearing shirt_b666.man_93...,1.0,What will the person do next? Guess the most l...,Wrong
13937,Prediction_T1_143,What will the person do next?,3N1I2,6.7,15.725,Take the pillow.,"[{'choice_id': 0, 'choice': 'Take the pillow.'...",In frame 0: man_ba84 wearing glass_e55c.man_ba...,3.0,What will the person do next? Guess the most l...,Wrong
13939,Prediction_T1_205,What will the person do next?,YHXU9,13.6,20.725,Throw the book.,"[{'choice_id': 0, 'choice': 'Wash the window.'...",In frame 0: curtain_3b44 covering window_14da....,2.0,What will the person do next? Guess the most l...,Wrong
13946,Prediction_T1_469,What will the person do next?,65UVU,0.0,6.150,Put down the box.,"[{'choice_id': 0, 'choice': 'Sit on the sofa/c...",In frame 0: handle_fa5c on door_358e.door_358e...,3.0,What will the person do next? Guess the most l...,Correct
13950,Prediction_T1_499,What will the person do next?,7TA23,6.6,16.300,Put down the shoe.,"[{'choice_id': 0, 'choice': 'Put down the bag....",In frame 0: man_4414 wearing shirt_dd33.man_44...,1.0,What will the person do next? Guess the most l...,Wrong


In [6]:
feasibility_ds = ds.loc[ds['question_id'].str.contains('Feasibility')]
feasibility_ds.name= 'Feasibility'
feasibility_ds.head()

,question_id,question,video_id,start,end,answer,choices,OIC_context,OIC_answer,OIC_question,Match
8428,Feasibility_T2_107,What else is the person able to do with the food?,YJ1KW,0.0,4.2,Take the food.,"[{'choice_id': 0, 'choice': 'Close the food.',...",In frame 0: man_2006 has hair_d536.man_2006 we...,1,What else is the person able to do with the fo...,Wrong
8429,Feasibility_T2_108,What else is the person able to do with the food?,YJ1KW,10.7,15.8,Take the food.,"[{'choice_id': 0, 'choice': 'Take the food.', ...",In frame 0: man_f86f has hair_4e40.man_f86f we...,3,What else is the person able to do with the fo...,Wrong
8430,Feasibility_T2_126,What else is the person able to do with the box?,9MNZ5,9.0,13.8,Close the box.,"[{'choice_id': 0, 'choice': 'Close the box.', ...",In frame 0: man_a164 wearing shirt_a75f.man_a1...,1,What else is the person able to do with the bo...,Correct
8431,Feasibility_T2_127,What else is the person able to do with the box?,9MNZ5,10.4,17.8,Close the box.,"[{'choice_id': 0, 'choice': 'Lie on the box.',...",In frame 0: man_5116 wearing shirt_aace.man_51...,4,What else is the person able to do with the bo...,Wrong
8432,Feasibility_T2_157,What else is the person able to do with the pi...,JTBXI,32.3,36.7,Put down the pillow.,"[{'choice_id': 0, 'choice': 'Close the pillow....",In frame 0: man_b94f has hair_88b6.man_b94f we...,4,What else is the person able to do with the pi...,Correct


In [7]:
for i in [ds, interaction_ds,sequence_ds,prediction_ds, feasibility_ds]:
  count = (i['Match'].value_counts()['Correct'])
  length = len(i)
  print('length of {} is: {}'.format(i.name, length))
  print('Correct: {}'.format(count))
  print('Correct match percentage for {} is: {}'.format(i.name, (count/length)*100))
  
  #ds.to_csv('all_cured.csv')

length of all is: 1140
Correct: 367
Correct match percentage for all is: 32.19298245614035
length of Interaction is: 624
Correct: 195
Correct match percentage for Interaction is: 31.25
length of Sequence is: 234
Correct: 73
Correct match percentage for Sequence is: 31.196581196581196
length of Prediction is: 89
Correct: 23
Correct match percentage for Prediction is: 25.842696629213485
length of Feasibility is: 193
Correct: 76
Correct match percentage for Feasibility is: 39.37823834196891
